In [1]:
from mercury import Chat, ChatInput, Message, Columns

In [9]:
left, right = Columns(2, border=True)

ColumnsBox(children=(ColumnOutput(layout=Layout(border_bottom='1px solid lightgray', border_left='1px solid li…

In [10]:
with left:
    left_chat = Chat()
with right:
    right_chat = Chat()

In [11]:
chat_input = ChatInput()

ChatInputWidget(cell_id='a085c253-3f15-4556-8523-3b56f78c04b4')

In [7]:
import asyncio

msg = chat_input.submitted.strip()

async def left_responder(user_text: str):
    await asyncio.sleep(1)  # simulate wait
    text = f"No: {user_text}"
    for ch in text:
        yield ch
        await asyncio.sleep(0.09)

async def right_responder(user_text: str):
    await asyncio.sleep(2)  # simulate wait
    text = f"Yes: {user_text}"
    for ch in text:
        yield ch
        await asyncio.sleep(0.05)

async def stream_responder(responder, user_text, bot_msg):
    async for chunk in responder(user_text):
        bot_msg.append_markdown(chunk)

async def handle_submit():
    if not msg:
        return

    # User prompt
    user_msg = Message(role="user", emoji="👤")
    user_msg.set_message(markdown=msg)
    left_chat.add(user_msg)
    right_chat.add(user_msg)

    # Left AI response shell
    left_bot_msg = Message(role="assistant", emoji="🤖")
    left_bot_msg.set_gradient_text("Thinking hard 🤔")
    left_chat.add(left_bot_msg)

    # Right AI response shell
    right_bot_msg = Message(role="assistant", emoji="🤖")
    right_bot_msg.set_bouncing_text("Thinking hard 🤔")
    right_chat.add(right_bot_msg)

    # Start both streams in parallel
    await asyncio.gather(
        stream_responder(left_responder, msg, left_bot_msg),
        stream_responder(right_responder, msg, right_bot_msg),
    )

    chat_input.submitted = ""


In [8]:
await handle_submit()